In [1]:
# Bibliotecas externas
import pyodbc
from datetime import datetime
import win32com.client
from datetime import datetime, timedelta

# Data de hoje
data_hoje = datetime.now()

# Primeira data do mês
primeira_data_do_mes = data_hoje.replace(day=1)

# Formatar as datas para o formato aaaa-mm-dd
data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
primeira_data_do_mes_formatada = primeira_data_do_mes.strftime("%Y-%m-%d")


def AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial):
    arrayParaVisualizarMelhor = [CodigoCarteira,Banco,Carteira,Filial]
    # Caminho do arquivo Excel
    caminho_excel = rf"C:\Users\t29969\Downloads\Dailys\{Filial}\Daily_{CodigoCarteira}_{Banco}_{Carteira}.xlsb"
    
    try:
        # Inicia uma instância do Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False

        # Abre o arquivo Excel
        workbook = excel.Workbooks.Open(Filename=caminho_excel)
        
        # Habilita as macros (caso estejam desabilitadas por padrão)
        excel.AutomationSecurity = 3
        sheet = workbook.Sheets('Aux_SQL') 
        sheet.Range("G2").Value = primeira_data_do_mes_formatada
        sheet.Range("H2").Value = data_hoje_formatada
        
        # Executa a macro
        excel.Application.Run('Atualiza_Tudo.Atu_tudo')
        print(f"Macro executada com sucesso para {arrayParaVisualizarMelhor}!")
        
        # Salva e fecha o arquivo
        workbook.Save()
        workbook.Close(SaveChanges=1)
        excel.Application.Quit()

    except Exception as e:
        print(f"Erro ao executar a macro para {arrayParaVisualizarMelhor}: {e}")
        del excel
    
    finally:
        # Libera a variável Excel
        del excel
        print(f"Processo concluído para {arrayParaVisualizarMelhor}.")

def ConsultaValida_ToAtualizaMacro():
    # String de conexão
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER=172.17.1.115;"
        f"DATABASE=dbdatastoreHouse;"
        f"UID=USR_MIS;"
        f"PWD=MIS@321"
    )

    # Tentar se conectar e processar
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()

        # Consulta SQL
        query = """  select	
                        a.CodigoCarteira,
                        b.Banco,
                        b.Carteira,
                        b.Filial,
                        a.msg
                        
                    from dbdatastorehouse.dbo.MapaDailyPesoToPy a with(nolock)
                    join dbdatastorehouse.dbo.dp_dailytopy b on a.CodigoCarteira = b.GrupoCliente
                    where a.DataAtualizacao = '2024-09-20' and b.Filial = 'Aviacao'
                """

        # Executar a consulta e armazenar os resultados
        cursor.execute(query)
        arrayCdCli = cursor.fetchall()

        # Fechar conexão
        cursor.close()
        conn.close()

        # Verificar se há resultados
        if arrayCdCli:
            for row in arrayCdCli:
                CodigoCarteira, Banco, Carteira, Filial, Msg = row

                # Tentar rodar a macro para cada resultado
                try:
                    AtualizaMacro(CodigoCarteira, Banco, Carteira, Filial)
                    print(f"Sucesso em: {row}")
                except Exception as e:
                    print(f"Erro em: {row}: {e}")
        else:
            print('Nenhum resultado encontrado.')

    except Exception as e:
        print(f"Erro ao conectar ou executar a consulta: {e}")

ConsultaValida_ToAtualizaMacro();

Macro executada com sucesso para [135, 'BV', 'MASSIFICADO', 'Aviacao']!
Processo concluído para [135, 'BV', 'MASSIFICADO', 'Aviacao'].
Sucesso em: (135, 'BV', 'MASSIFICADO', 'Aviacao', 'Enviar')
Macro executada com sucesso para [32, 'BV', 'REMANESCENTE', 'Aviacao']!
Processo concluído para [32, 'BV', 'REMANESCENTE', 'Aviacao'].
Sucesso em: (32, 'BV', 'REMANESCENTE', 'Aviacao', 'Enviar')
Macro executada com sucesso para [137, 'BV', 'CARTOES', 'Aviacao']!
Processo concluído para [137, 'BV', 'CARTOES', 'Aviacao'].
Sucesso em: (137, 'BV', 'CARTOES', 'Aviacao', 'NÃO enviar')
Macro executada com sucesso para [138, 'BV', 'CL', 'Aviacao']!
Processo concluído para [138, 'BV', 'CL', 'Aviacao'].
Sucesso em: (138, 'BV', 'CL', 'Aviacao', 'NÃO enviar')
Macro executada com sucesso para [139, 'BV', 'CP', 'Aviacao']!
Processo concluído para [139, 'BV', 'CP', 'Aviacao'].
Sucesso em: (139, 'BV', 'CP', 'Aviacao', 'NÃO enviar')
Macro executada com sucesso para [621, 'BV', 'DAYCOVAL', 'Aviacao']!
Processo c

In [28]:
from datetime import datetime, timedelta

# Data de hoje
data_hoje = datetime.now()

# Primeira data do mês
primeira_data_do_mes = data_hoje.replace(day=1)

# Formatar as datas para o formato aaaa-mm-dd
data_hoje_formatada = data_hoje.strftime("%Y-%m-%d")
primeira_data_do_mes_formatada = primeira_data_do_mes.strftime("%Y-%m-%d")

print(data_hoje_formatada)
print(primeira_data_do_mes_formatada)

2024-09-24
2024-09-01
